Assignment 8: Cody Taylor

In [1]:
import pygame
import moderngl
import numpy as np
import glm
from PIL import Image
from loadModelUsingAssimp_V1_Edited import getObjectDataList

pygame.init()

# Window setup
width = 600
height = 600
screen = pygame.display.set_mode((width, height), flags=pygame.OPENGL | pygame.RESIZABLE)
pygame.display.set_caption(title="Assignment 8: Cody Taylor")
gl = moderngl.get_context()

# Vertex shader
vertex_shader_code = '''
    #version 330 core
    in vec4 in_position;
    in vec2 in_texCoord;
    in vec3 in_normal;
    
    uniform mat4 view;
    uniform mat4 perspective;
    uniform mat4 model;
    
    out vec2 fragTexCoord;
    out vec3 fragNormal;
    
    void main()
    {
        fragTexCoord = in_texCoord;
        fragNormal = mat3(transpose(inverse(model))) * in_normal;
        gl_Position = perspective * view * model * in_position;
    }
'''

# Fragment shader
fragment_shader_code = '''
    #version 330 core
    
    in vec2 fragTexCoord;
    in vec3 fragNormal;
    out vec4 color;
    
    uniform sampler2D textureIn;
    uniform vec3 lightDir;
    
    void main()
    {
        // Get material color from texture
        vec4 materialColor = texture(textureIn, fragTexCoord);
        
        // Normalize the normal vector
        vec3 normal = normalize(fragNormal);
        
        // Normalize the light direction (pointing FROM light TO fragment)
        vec3 lightDirection = normalize(lightDir);
        
        // Compute diffuse lighting
        float diffuse = max(dot(normal, lightDirection), 0.0);
        
        // Add ambient light
        float ambient = 0.3;
        
        // Combine lighting
        vec3 result = (ambient + diffuse) * materialColor.rgb;
        color = vec4(result, materialColor.a);
    }
'''
program = gl.program(vertex_shader=vertex_shader_code, fragment_shader=fragment_shader_code)
vbas = []
textures = []

# Takes in object file. Arranges the vertices and texture coordinates.
shape_data_list, face_data_list, bound, tex_names, scene = getObjectDataList("./mario_obj/scene.gltf", verbose = False)
for i, (shape_data, face_data) in enumerate(zip(shape_data_list, face_data_list)):
    vertex_buffer = gl.buffer(shape_data.tobytes())
    face_buffer = gl.buffer(face_data.tobytes())

    vba = gl.vertex_array(program, [(vertex_buffer, "3f 2f 3f", "in_position", "in_texCoord", "in_normal")], index_buffer = face_buffer)
    vbas.append(vba)

    texture_path = tex_names[i]
    image = Image.open(texture_path).transpose(Image.FLIP_TOP_BOTTOM).convert("RGB")
    texture = gl.texture(image.size, 3, image.tobytes())
    texture.build_mipmaps()
    textures.append(texture)
    print(textures[i])

gl.enable(moderngl.DEPTH_TEST)

# Sets the look at point to the origin and the up vector.
lookAtPoint = glm.vec3(bound.center)
upVector = glm.vec3(0.0, 1.0, 0.0)

# Sets the camera distance and the viewing angle. Also sets the initial y and z values of the camera.
camera_distance = 2.0 * bound.radius
y_angle = 90
y_initial = camera_distance * np.cos(np.radians(y_angle))
z_initial = camera_distance * np.sin(np.radians(y_angle))

# Sets the fov, near plane, and far plane of the camera.
fov = 45.0
near_plane = bound.radius * 0.1  
far_plane = bound.radius * 10.0 

gl.enable(moderngl.DEPTH_TEST)
gl.disable(moderngl.CULL_FACE)

# Main loop
running = True
clock = pygame.time.Clock()
rotation_angle = 0.0
rotation_speed = 0.5 

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key == 27):
            running = False
        if event.type == pygame.VIDEORESIZE:
            width, height = event.size
    
    gl.clear(0.3, 0.3, 0.4)
    gl.clear(depth=True)
    
    # Finds the viewing point of the camera based off of it's current rotation.
    angle_rad = np.radians(rotation_angle)
    camera_distance = 2.0 * bound.radius

    theta = np.radians(y_angle) 
    phi = angle_rad 

    x = camera_distance * np.sin(theta) * np.sin(phi)
    y = bound.center.y
    z = camera_distance * np.sin(theta) * np.cos(phi)

    eyePoint = glm.vec3(x, y, z)

    # Creates the viewing matrix and perspective matrix to send to the shader.
    view_matrix = glm.lookAt(eyePoint, lookAtPoint, upVector)
    aspect_ratio = width / height
    perspective_matrix = glm.perspective(glm.radians(fov), aspect_ratio, near_plane, far_plane)
    
    model_matrix = glm.rotate(glm.mat4(1.0), glm.radians(-90.0), glm.vec3(1.0, 0.0, 0.0))

    program['view'].write(np.array(view_matrix, dtype='f4').T.tobytes())
    program['perspective'].write(np.array(perspective_matrix, dtype='f4').T.tobytes())
    program['model'].write(np.array(model_matrix, dtype='f4').T.tobytes())
    
    program['lightDir'].value = (eyePoint.x, eyePoint.y, eyePoint.z)

    for i, obj in enumerate(vbas):
        if i < len(textures):
            textures[i].use(location=0)
            program['textureIn'].value = 0
        
        # Render this object
        obj.render(moderngl.TRIANGLES)
    
    pygame.display.flip()
    
    rotation_angle += rotation_speed
    if rotation_angle >= 360.0:
        rotation_angle = 0
    
    clock.tick(60)

pygame.quit()

pygame 2.6.1 (SDL 2.28.4, Python 3.13.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


ModuleNotFoundError: No module named 'LoadObject'